# Housing Analysis for Phoenix, Arizona 

In [61]:
# import libraries 
import pandas as pd 
import hvplot.pandas 
from pathlib import Path 
import datetime as dt
import altair as alt 
import requests
import json 

In [2]:
# import the housing units data from 2012 - 2022
phx_housing_units_df = pd.read_csv(Path('Resources\housing_inventory.csv'))

# review the first and last five rows of the dataframe 
display(phx_housing_units_df.head())
display(phx_housing_units_df.tail())

,Months,PHX_Housing_Inventory
0,February 2012,5805
1,March 2012,5453
2,April 2012,5355
3,May 2012,5266
4,June 2012,5362


,Months,PHX_Housing_Inventory
121,March 2022,3238
122,April 2022,3263
123,May 2022,2903
124,June 2022,2349
125,July 2022,2283


In [3]:
# create a visual aggregation exploring the housing units by year in Phoenix
phx_housing_units_df.hvplot.bar(
    x = 'Months', 
    xlabel = 'Period', 
    ylabel = 'Total Housing Inventory', 
    title = 'Total Housing Inventory in Phoenix, Arizona from 2012 to 2022', 
    yformatter = '%.0f',
    rot = 90, 
    ylim = (1000,9000),
    height = 500, 
    width = 1250
).opts(
    color = 'blue',
    hover_color = 'orange'
)

:Bars   [Months]   ( PHX_Housing_Inventory)

In [48]:
# read in the homes sold by year data
homes_sold_data = Path("Resources/homes_sold.csv")
homes_sold_df = pd.read_csv(homes_sold_data)

# drop extra columns
homes_sold_df.drop(['Day of Year', 'Year of Period End', 'Period Begin', 'adjusted_average_homes_sold_yoy', 'Average Homes Sold Yoy (tooltip)'], 
                                    axis=1, inplace=True)

homes_sold_df.rename(columns = {'Period End':'Period_End'}, inplace = True)

# sort dates
homes_sold_df['Period End'] = pd.to_datetime(homes_sold_df.Period_End)
homes_sold_df.sort_values(['Period End'], ascending = False)
homes_sold_df.drop(['Period_End'], axis = 1, inplace = True)

display(homes_sold_df.head())
display(homes_sold_df.tail())

,adjusted_average_homes_sold,Period End
0,"1,879",2022-01-02
1,"1,931",2021-01-03
2,"1,551",2020-01-05
3,"1,389",2019-01-06
4,"1,796",2022-01-09


,adjusted_average_homes_sold,Period End
186,"2,128",2020-12-20
187,"1,758",2019-12-22
188,"2,024",2021-12-26
189,"2,103",2020-12-27
190,"1,655",2019-12-29


In [41]:
# clean the data
homes_sold_df['adjusted_average_homes_sold']=homes_sold_df['adjusted_average_homes_sold'].str.replace(',','').astype(float)

In [42]:
homes_sold_df.dtypes

adjusted_average_homes_sold           float64
Period End                     datetime64[ns]
dtype: object

In [43]:
# create visual 
homes_sold_df.hvplot.line(
    x="Period End", 
    xlabel = "Period End", 
    ylabel = "Adjusted Average Homes Sold",
    title = "Adjusted Average Homes Sold by Year",
    rot = 90,
    width = 1200, 
    height = 500
    )

:Curve   [Period End]   (adjusted_average_homes_sold)

In [50]:
# pulling data from investor purchases csv and creating dataframe
investor_purchases_data = Path("Resources/investor_purchases_market_share.csv")
investor_purchases_df = pd.read_csv(investor_purchases_data)
investor_purchases_df

# removing commas and changing datatype to 'float'
investor_purchases_df['All Home Sales']=investor_purchases_df['All Home Sales'].str.replace(',','').astype(float)
investor_purchases_df['Investor Purchases']=investor_purchases_df['Investor Purchases'].str.replace(',','').astype(float)

# setting 'Quarter' column to datetime format
investor_purchases_df["Quarter"] = pd.to_datetime(investor_purchases_df["Quarter"])

display(investor_purchases_df.head())
display(investor_purchases_df.tail())

,Quarter,Redfin Metro,All Home Sales,Investor Market Share,Investor Purchases,Max Investor Market Share
0,2000-01-01,"Phoenix, AZ",16772.0,0.102969,1727.0,NaN
1,2000-01-01,"Phoenix, AZ",16772.0,0.102969,1727.0,NaN
2,2000-04-01,"Phoenix, AZ",21055.0,0.090905,1914.0,NaN
3,2000-04-01,"Phoenix, AZ",21055.0,0.090905,1914.0,NaN
4,2000-07-01,"Phoenix, AZ",17476.0,0.099393,1737.0,NaN


,Quarter,Redfin Metro,All Home Sales,Investor Market Share,Investor Purchases,Max Investor Market Share
175,2021-10-01,"Phoenix, AZ",27068.0,0.294591,7974.0,NaN
176,2022-01-01,"Phoenix, AZ",24941.0,0.291688,7275.0,NaN
177,2022-01-01,"Phoenix, AZ",24941.0,0.291688,7275.0,NaN
178,2022-04-01,"Phoenix, AZ",26350.0,0.312448,8233.0,0.312448
179,2022-04-01,"Phoenix, AZ",26350.0,0.312448,8233.0,0.312448


In [54]:
# plotting altair bar chart utilizing transform_fold feature (overlaying and creating legend)
chart_1 = alt.Chart(investor_purchases_df).mark_bar().transform_fold(
    ['All Home Sales', 'Investor Purchases'], 
    as_=['variable', 'value']
).encode(
    x='Quarter:T',
    y='max(value):Q',
    color='variable:N'
).properties(
    title='Total Sales and Investor Purchases by Year',
    width=800,
    height=400)

chart_1

alt.Chart(...)

In [55]:
chart_2 = alt.Chart(investor_purchases_df).mark_bar(color="yellow").encode(
    x="Quarter",
    y="Investor Purchases",
    text='Investor Purchases'
).properties(
    width=600,
    height=400)

# create a layered chart 
alt.layer(chart_1,chart_2)

alt.LayerChart(...)

In [56]:
# import the phoenix housing listings data
phx_housing_listings_df = pd.read_csv(
                                    Path('Resources/Phoenix_listings.csv'), 
                                    index_col = 'rawAddress')

phx_housing_listings_df.drop(['Unnamed: 0', 'county', 'bedrooms', 'bathrooms', 'yearBuilt', 'lotSize',
                            'addressLine1', 'city', 'state', 'formattedAddress', 'lastSeen', 'listedDate', 'status', 
                            'removedDate', 'daysOnMarket', 'createdDate', 'id', 'addressLine2' ], 
                            axis=1, inplace=True)

phx_housing_listings_df = phx_housing_listings_df.dropna()

# review the first and last five rows of the dataframe 
display(phx_housing_listings_df.head())

,squareFootage,propertyType,price,zipCode,latitude,longitude
rawAddress,,,,,,
"3004 W Running Deer Trl, Phoenix, AZ 85083",1524,Single Family,479000,85083,33.738370,-112.123658
"2217 N 23rd Pl, Phoenix, AZ 85006",1546,Single Family,381000,85006,33.471916,-112.031007
"2553 E Pueblo Ave, Phoenix, AZ 85040",2484,Duplex-Triplex,699000,85040,33.407599,-112.024221
"4625 N 78th Dr, Phoenix, AZ 85033",1682,Single Family,410000,85033,33.504775,-112.227614
"2020 N 78th Ave, Phoenix, AZ 85035",1383,Single Family,366000,85035,33.469990,-112.227193


In [57]:
# Create a plot to analyze housing listings info 
phx_housing_listings_df.hvplot.points(
    'longitude',
    'latitude', 
    geo = True, 
    frame_width = 700, 
    frame_height = 500, 
    hover_cols = 'rawAddress',
    tiles = 'OSM',
    color = 'price',
    size = 'squareFootage',
    title = 'Phoenix Housing Property Listings -- Interactive Heat Map'
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (price,squareFootage,rawAddress)

In [59]:
# import the phoenix rental housing listings data
phx_housing_rental_listings_df = pd.read_csv(
                                    Path('Resources/Phoenix_rent_listings.csv'), 
                                    index_col = 'rawAddress')

phx_housing_rental_listings_df.drop(['Unnamed', 'addressLine1', 'formattedAddress', 'city', 'state', 'zipCode', 'county', 
                                    'bedrooms', 'bathrooms', 'createdDate', 'lastSeen', 'listedDate', 'status', 
                                    'removedDate', 'daysOnMarket', 'id', 'addressLine2', 'yearBuilt'], 
                                    axis=1, inplace=True)

phx_housing_rental_listings_df = phx_housing_rental_listings_df.dropna()

# review the first and last five rows of the dataframe 
display(phx_housing_rental_listings_df.head())

,price,propertyType,latitude,longitude,squareFootage
rawAddress,,,,,
"7234 S 39th Dr # A2, Phoenix, AZ 85041",2105,Single Family,33.380651,-112.143215,2093.0
"1129 E Garfield St E Garfield St Unit ET1129, Phoenix, AZ 85006",1700,Apartment,33.457455,-112.057589,600.0
"2222 E Wayland Dr, Phoenix, AZ 85040",2230,Single Family,33.394604,-112.033668,1743.0
"3816 N 83rd Ave, Phoenix, AZ 85033",1340,Apartment,33.491358,-112.239661,770.0
"4046 W Northview Ave, Phoenix, AZ 85051",1900,Single Family,33.543607,-112.146675,1620.0


In [60]:
# Create a plot to analyze rental housing listings info 
phx_housing_rental_listings_df.hvplot.points(
    'longitude',
    'latitude', 
    geo = True, 
    frame_width = 700, 
    frame_height = 500, 
    hover_cols = 'rawAddress',
    tiles = 'OSM',
    color = 'price',
    size = 'squareFootage',
    title = 'Phoenix Rental Housing Property Listings -- Interactive Heat Map'
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (price,squareFootage,rawAddress)

In [66]:
# pull data from redfin API 
url = "https://unofficial-redfin.p.rapidapi.com/properties/list"

querystring = {"region_id":"14240","region_type":"6","uipt":"1,2,3,7,8","status":"9","sf":"1,2,5,6,7","num_homes":"300","sold_within_days":"30"}

headers = {
	"X-RapidAPI-Key": "b6a21fcaa0msh2e9de61afa99433p14003ajsnd33bf08e42ad",
	"X-RapidAPI-Host": "unofficial-redfin.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

data = response.json()


print(f"Start API acquisition")
print(f"------------&&&&-------------------")

# type(data)
# print(json.dumps(data, indent=4))

Start API acquisition
------------&&&&-------------------


In [63]:
property_id = []
property_type = []
beds = []
baths = []
price_info = []
latitude = [] 
longitude = [] 

for homeData in data['homes']:
    property_id.append(homeData['homeData']['propertyId'])
    property_type.append(homeData['homeData']['propertyType'])
    #beds.append(homeData['homeData']['beds']['value'])
    #baths.append(homeData['homeData']['baths']['value'])
    price_info.append(homeData['homeData']['priceInfo']['amount']['value'])
    latitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['latitude'])
    longitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['longitude'])

In [64]:
data_dict = {"Property_ID": property_id, "Property_Type": property_type, "Price Info": price_info, "Latitude": latitude, "Longitude": longitude}
data_df = pd.DataFrame(data_dict)
data_df

,Property_ID,Property_Type,Price Info,Latitude,Longitude
0,27959747,SINGLE_FAMILY_RESIDENTIAL,590000,33.720061,-112.146794
1,26988109,TOWNHOUSE,346000,33.646273,-112.156695
2,27277865,SINGLE_FAMILY_RESIDENTIAL,455500,33.631259,-112.084501
3,28139777,SINGLE_FAMILY_RESIDENTIAL,450000,33.417653,-112.245364
4,27650441,SINGLE_FAMILY_RESIDENTIAL,455000,33.715647,-112.201680
...,...,...,...,...,...
295,28060421,CONDO_COOP,321000,33.630895,-112.121145
296,28122395,SINGLE_FAMILY_RESIDENTIAL,560000,33.634864,-112.127699
297,27250129,SINGLE_FAMILY_RESIDENTIAL,480000,33.634062,-112.120899
298,26922907,SINGLE_FAMILY_RESIDENTIAL,425000,33.634764,-112.125021


In [65]:
# create visual based on property type 
data_df.hvplot.points(
    'Longitude',
    'Latitude', 
    geo = True, 
    frame_width = 700, 
    frame_height = 500, 
    tiles = 'OSM',
    color = 'Property_Type',
    hover_cols = 'Price Info',
    title = 'Phoenix Housing Property Type Listings'
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Property_Type,Price Info)